In [1]:
import sys
sys.path.append('../src')

In [2]:
import pickle
import os
import collections
import numpy as np
import math
import encoder_decoder_uttt as ed
from utictactoe_board import board as c_board
import copy
import torch
import torch.multiprocessing as mp
from alpha_net_uttt import ConnectNet
import datetime
import logging
from tqdm import tqdm

In [3]:
net = ConnectNet()

In [4]:
cuda = torch.cuda.is_available()

In [5]:
net.cuda()

ConnectNet(
  (conv): ConvBlock(
    (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_0): ResBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_1): ResBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  

In [6]:
with torch.no_grad():
    current_board = c_board()
    checkmate = False
    board_state = copy.deepcopy(ed.encode_board(current_board))

In [9]:
root = UCTNode(current_board, move=None, parent=DummyNode())

In [14]:
leaf = root.select_leaf()

In [15]:
encoded_s = ed.encode_board(leaf.game)
encoded_s = encoded_s.transpose(2,0,1)
encoded_s = torch.from_numpy(encoded_s).float().cuda()

In [16]:
child_priors, value_estimate = net(encoded_s)

In [17]:
child_priors

tensor([[0.0113, 0.0064, 0.0076, 0.0123, 0.0167, 0.0103, 0.0115, 0.0106, 0.0152,
         0.0114, 0.0097, 0.0123, 0.0084, 0.0258, 0.0175, 0.0090, 0.0073, 0.0045,
         0.0178, 0.0372, 0.0110, 0.0083, 0.0079, 0.0071, 0.0168, 0.0101, 0.0083,
         0.0076, 0.0083, 0.0180, 0.0155, 0.0087, 0.0071, 0.0135, 0.0170, 0.0087,
         0.0065, 0.0158, 0.0210, 0.0129, 0.0068, 0.0075, 0.0081, 0.0076, 0.0175,
         0.0050, 0.0143, 0.0105, 0.0251, 0.0097, 0.0181, 0.0129, 0.0187, 0.0224,
         0.0110, 0.0144, 0.0223, 0.0066, 0.0189, 0.0089, 0.0099, 0.0066, 0.0134,
         0.0064, 0.0189, 0.0087, 0.0071, 0.0080, 0.0111, 0.0139, 0.0181, 0.0057,
         0.0103, 0.0042, 0.0178, 0.0069, 0.0113, 0.0072, 0.0128, 0.0165, 0.0259]],
       device='cuda:0', grad_fn=<ExpBackward>)

In [18]:
value_estimate

tensor([[-0.1647]], device='cuda:0', grad_fn=<TanhBackward>)

In [19]:
child_priors = child_priors.detach().cpu().numpy().reshape(-1)
value_estimate = value_estimate.item()

In [20]:
child_priors

array([0.01127057, 0.00641429, 0.00762172, 0.012283  , 0.01674675,
       0.01034931, 0.01145779, 0.01059688, 0.01524623, 0.01144383,
       0.00971424, 0.01228233, 0.00839466, 0.02583111, 0.01750324,
       0.00898575, 0.00730001, 0.00446371, 0.0177677 , 0.03717604,
       0.01102446, 0.00833959, 0.00794205, 0.00714903, 0.01684039,
       0.0100558 , 0.00825702, 0.00756053, 0.00825731, 0.01798186,
       0.01550732, 0.00871139, 0.00706415, 0.01352002, 0.0170088 ,
       0.00872987, 0.00650245, 0.01579166, 0.02098247, 0.01290449,
       0.00679153, 0.00745083, 0.00809259, 0.00755052, 0.01748762,
       0.00498794, 0.01433045, 0.01052347, 0.02508257, 0.00971188,
       0.01807436, 0.0129144 , 0.0187206 , 0.02244591, 0.01103003,
       0.01443   , 0.02225366, 0.00656632, 0.01892641, 0.00894512,
       0.00993539, 0.00657042, 0.01340808, 0.0064424 , 0.01889094,
       0.00870549, 0.00709207, 0.00800012, 0.01107017, 0.01385787,
       0.01813106, 0.00573949, 0.01027886, 0.00417075, 0.01780

In [11]:
leaf.game.check_winner()

False

In [12]:
leaf.game.actions()

[(6, 6), (6, 7), (6, 8), (7, 6), (7, 7), (7, 8), (8, 6), (8, 7), (8, 8)]

In [25]:
valid_moves = leaf.game.actions()

In [26]:
child_priors[valid_moves]

/home/sf/Applications/anaconda3/envs/torchenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


IndexError: too many indices for array

In [22]:
c_p = child_priors
action_idxs = leaf.game.actions()
c_p[[i for i in range(len(child_priors)) if i not in action_idxs]] = 0.000000000

In [23]:
c_p

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [24]:
child_priors[(6,6)]

IndexError: too many indices for array

In [21]:
leaf.expand(child_priors)

/home/sf/Applications/anaconda3/envs/torchenv/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


IndexError: too many indices for array

In [21]:
def return_winner(winner):
    if winner == "O" or winner == "X":
        print(1)
        return True
    elif winner == "tie":
        print(2)
        return winner
    else:
        return False

In [22]:
winner = None

In [20]:
winner == "O" or "X"

'X'

In [23]:
return_winner(winner=None)

False

In [30]:
leaf.game.mega_board

array([[' ', ' ', ' '],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U32')

In [20]:
leaf.game.current_board

array([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']], dtype='<U32')

In [23]:
y = check_ninebox(leaf.game.mega_board)

In [24]:
y

In [21]:
def check_ninebox(mat):
    """
    This function can be use to check the small nine-box \
            or the large nine-box.
    input: mat: a matrix of 3x3
    """
    # look for rows, columns, diagnols
    for i in range(3):
        if mat[i,0] == mat[i,1] == mat[i,2] != ' ':
            return mat[i,0]
        if mat[0,i] == mat[1,i] == mat[2,i] != ' ':
            return mat[0,i]
    if mat[0,0] == mat[1,1] == mat[2,2] != ' ':
        return mat[0,0]
    if mat[0,2] == mat[1,1] == mat[2,0] != ' ':
        return mat[1,1]

    # look for tie
    count = 0
    for i in range(3):
        for j in range(3):
            if mat[i,j] != ' ':
                count += 1
    if count == 9:
        return 'tie'
    else:
        return None

In [7]:
class UCTNode():
    def __init__(self, game, move, parent=None):
        self.game = game # state s
        self.move = move # action index
        self.is_expanded = False
        self.parent = parent  
        self.children = {}
        self.child_priors = np.zeros([81], dtype=np.float32)
        self.child_total_value = np.zeros([81], dtype=np.float32)
        self.child_number_visits = np.zeros([81], dtype=np.float32)
        self.action_idxes = []
        
    @property
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value
    
    @property
    def total_value(self):
        return self.parent.child_total_value[self.move]
    
    @total_value.setter
    def total_value(self, value):
        self.parent.child_total_value[self.move] = value
    
    def child_Q(self):
        return self.child_total_value / (1 + self.child_number_visits)
    
    def child_U(self):
        return math.sqrt(self.number_visits) * (
            abs(self.child_priors) / (1 + self.child_number_visits))
    
    def best_child(self):
        if self.action_idxes != []:
            bestmove = self.child_Q() + self.child_U()
            bestmove = self.action_idxes[np.argmax(bestmove[self.action_idxes])]
        else:
            bestmove = np.argmax(self.child_Q() + self.child_U())
        return bestmove
    
    def select_leaf(self):
        current = self
        while current.is_expanded:
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current
    
    def add_dirichlet_noise(self,action_idxs,child_priors):
        valid_child_priors = child_priors[action_idxs] # select only legal moves entries in child_priors array
        valid_child_priors = 0.75*valid_child_priors + 0.25*np.random.dirichlet(np.zeros([len(valid_child_priors)], \
                                                                                          dtype=np.float32)+192)
        child_priors[action_idxs] = valid_child_priors
        return child_priors
    
    def expand(self, child_priors):
        self.is_expanded = True
        action_idxs = self.game.actions()
        c_p = child_priors
        if action_idxs == []:
            self.is_expanded = False
        self.action_idxes = action_idxs
        c_p[[i for i in range(len(child_priors)) if i not in action_idxs]] = 0.000000000 # mask all illegal actions
        if self.parent.parent == None: # add dirichlet noise to child_priors in root node
            c_p = self.add_dirichlet_noise(action_idxs,c_p)
        self.child_priors = c_p
    
    def decode_n_move_pieces(self,board,move):
        board.drop_piece(move)
        return board
            
    def maybe_add_child(self, move):
        if move not in self.children:
            copy_board = copy.deepcopy(self.game) # make copy of board
            copy_board = self.decode_n_move_pieces(copy_board,move)
            self.children[move] = UCTNode(
              copy_board, move, parent=self)
        return self.children[move]
    
    def backup(self, value_estimate: float):
        current = self
        while current.parent is not None:
            current.number_visits += 1
            if current.game.player == 1: # same as current.parent.game.player = 0
                current.total_value += (1*value_estimate) # value estimate +1 = O wins
            elif current.game.player == 0: # same as current.parent.game.player = 1
                current.total_value += (-1*value_estimate)
            current = current.parent

In [8]:
class DummyNode(object):
    def __init__(self):
        self.parent = None
        self.child_total_value = collections.defaultdict(float)
        self.child_number_visits = collections.defaultdict(float)